### Google calendar quickstart code

In [5]:
from __future__ import print_function
import datetime
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def skrrt():
    # file token.json only exists if workflow run thru once already
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            # Save credentials for next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
    service = build('calendar', 'v3', credentials=creds)
    
    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' #'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now, 
                                          maxResults=10, singleEvents=True,
                                         orderBy='startTime').execute()
    events = events_result.get('items', [])
    
    if not events:
        print('No upcoming events found')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        # Why does this work in this particular way?
        print(start, event['summary'])

skrrt()

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

### What's next?

We're gonna grab my calendar and put it inside a data structure so that we can modify it after.

In [ ]:
def initSetup():
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            # Save credentials for next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
    return build('calendar', 'v3', credentials=creds)

now = datetime.datetime.utcnow().isoformat() + 'Z' # Apparently Z indicates UTC time

# This is where i'm gonna call the service and then put it in some sort of structure
events_result = service.events().list(calendarId='primary', timeMin=now,
                                     maxResults=10, singleEvents=True,
                                     orderBy='startTime').execute()
events = events_result.get('items', []) # So apparently each event is stored
# as an individual object. It's an iterable so I can iterate thru everything
    
# Next i'm going to do something with each event



### What to do next

I should probably make some objects to make my life easier. Maybe wrap the
google objects in a general event object wrapper and then get some accessor
methods to generalize everything. I would have to access the:
- Name
- Start date
- End date

Then there might be some google particulars but I can deal with those another time.

In [2]:
class Event:
    def setName(self, name):
        self.name = name
    def setStartDate(self, date):
        self.start_date = date
    def getStartDate(self):
        return self.start_date
    def setEndDate(self, date):
        self.end_date = date
    def getEndDate(self):
        return self.end_date

class GoogleEvent(Event):
    def __init__(self, google_event):
        self.name = google_event.description
        self.gObj = google_event
    def getName(self):
        return self.name
    def getStartDate(self):
        self.setStartDate(self.gObj['start'].get('dateTime', self.gObj['start'].get('date')))
        return self.getStartDate()
    def getEndDate(self):
        self.setEndDate(self.gObj['end'].get('dateTime', self.gObj['end'].get('date')))
        return self.getEndDate()

def processEvents(calendar_name):
    service = initSetup() # Get the thing started
    # Next up: list all of the events
    now = datetime.datetime.utcnow().isoformat() + 'Z'
    events_result = service.events().list(calendarId="primary", timeMin=now, 
    maxResults=10, singleevents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])
    geevents = []
    for event in events:
        geevents.extend(GoogleEvent(event))
    # This should process the events into the google objects.
    # TODO: Process the events into the correct matricies and do some cool math
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-d63a77c1bfd1>, line 26)